# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [7]:
# Define the target set y using the credit_ranking column
y =loans_df["credit_ranking"]

# Display a sample of y 
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [8]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [9]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale the features training and testing datasets
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [11]:
# Define the the number of inputs (features) to the model
input_nodes = len(X_train_scaled[0])

# Review the number of features
input_nodes

11

In [12]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes1 = 8

# Define the number of hidden nodes for the second hidden layer
hidden_nodes2 = 5

# Define the number of neurons in the output layer
hidden_output = 1

In [14]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=hidden_nodes1, activation='relu', input_dim=input_nodes))

# Add the second hidden layer

model.add(Dense(units=hidden_nodes2, activation='relu'))
# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=hidden_output, activation='sigmoid'))

In [15]:
# Display the Sequential model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 96        
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 147 (588.00 Byte)
Trainable params: 147 (588.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [16]:
# Compile the Sequential model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Fit the model using 50 epochs and the training data
fit_model = model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50


38/38 [==============================] - 1s 889us/step - loss: 0.7055 - accuracy: 0.4812
Epoch 2/50
38/38 [==============================] - 0s 622us/step - loss: 0.6884 - accuracy: 0.5446
Epoch 3/50
38/38 [==============================] - 0s 586us/step - loss: 0.6769 - accuracy: 0.5972
Epoch 4/50
38/38 [==============================] - 0s 595us/step - loss: 0.6643 - accuracy: 0.6205
Epoch 5/50
38/38 [==============================] - 0s 585us/step - loss: 0.6480 - accuracy: 0.6514
Epoch 6/50
38/38 [==============================] - 0s 613us/step - loss: 0.6275 - accuracy: 0.6697
Epoch 7/50
38/38 [==============================] - 0s 568us/step - loss: 0.6049 - accuracy: 0.7189
Epoch 8/50
38/38 [==============================] - 0s 598us/step - loss: 0.5825 - accuracy: 0.7264
Epoch 9/50
38/38 [==============================] - 0s 582us/step - loss: 0.5631 - accuracy: 0.7348
Epoch 10/50
38/38 [==============================] - 0s 609us/step - loss: 0.5481 - accuracy: 0.74

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [18]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 [==============================] - 0s 709us/step - loss: 0.5135 - accuracy: 0.7450
Loss: 0.5135497450828552, Accuracy: 0.7450000047683716


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [19]:
# Set the model's file path
path = Path("student_loans.keras")

# Export your model to a keras file
model.save(path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [20]:
# Set the model's file path
path = Path("student_loans.keras")

# Load the model to a new object
model = tf.keras.models.load_model(path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [21]:
# Make predictions with the test data
predictions = model.predict(X_test_scaled)

# Display a sample of the predictions
predictions[:5]

13/13 [==============================] - 0s 542us/step


array([[0.5006886],
       [0.3140381],
       [0.8164274],
       [0.6757468],
       [0.9781096]], dtype=float32)

In [22]:
# Save the predictions to a DataFrame and round the predictions to binary results
results = pd.DataFrame(predictions, columns=["predictions"])
results["predictions"] = results["predictions"].apply(lambda x: 1 if x >= 0.5 else 0)
results.head()

,predictions
0,1
1,0
2,1
3,1
4,1


### Step 4: Display a classification report with the y test data and predictions

In [23]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, results["predictions"]))

              precision    recall  f1-score   support

           0       0.70      0.79      0.74       188
           1       0.79      0.71      0.75       212

    accuracy                           0.74       400
   macro avg       0.75      0.75      0.74       400
weighted avg       0.75      0.74      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

1. To recommend loans from students you would need data from there bank, location, school info i.e.. gpa. These all have correlations to determine an appropriate loan option for the studdent.
2. Context based filtering would be best in this case. Certain indivuals qualities do not necissarily apply to each correlation feature. Context based filtering would help ensure it is not making decisions on feature the student would not have control over.
3. Based off of location there could be unfair loan recommendations to low income areas. It could filter through and look over students who are deserving of a good loan but the location correlation could oversee people of lower income areas.
